In [1]:
import sys
sys.path.append('./KPConv-PyTorch/')

In [2]:
from utils.config import Config
from datasets.APRPointCloud import APRPointCloudDataset, APRPointCloudCollate, APRPointCloudCustomBatch, APRPointCloudSampler
from models.architectures import KPFCNN
from torch.utils.data import DataLoader
from utils.trainer import ModelTrainer
import numpy as np
import torch

In [3]:
class APRPointCloudConfig(Config):
    """
    Override the parameters you want to modify for this dataset
    """

    ####################
    # Dataset parameters
    ####################

    # Dataset name
    dataset = 'APRPointCloud'

    # Number of classes in the dataset (This value is overwritten by dataset class when Initializating dataset).
    num_classes = None

    # Type of task performed on this dataset (also overwritten)
    dataset_task = 'cloud_segmentation'

    # Number of CPU threads for the input pipeline
    input_threads = 0

    #########################
    # Architecture definition
    #########################

    # Define layers
    architecture = ['simple',
                    'resnetb',
                    'resnetb_strided',
                    'resnetb',
                    'resnetb',
                    'resnetb_strided',
                    'resnetb_deformable',
                    'resnetb_deformable',
                    'resnetb_deformable_strided',
                    'resnetb_deformable',
                    'resnetb_deformable',
                    'resnetb_deformable_strided',
                    'resnetb_deformable',
                    'resnetb_deformable',
                    'nearest_upsample',
                    'unary',
                    'nearest_upsample',
                    'unary',
                    'nearest_upsample',
                    'unary',
                    'nearest_upsample',
                    'unary']

    ###################
    # KPConv parameters
    ###################

    # Radius of the input sphere
    in_radius = 3.0 #2.0

    # Number of kernel points
    num_kernel_points = 15

    # Size of the first subsampling grid in meter
    first_subsampling_dl = 0.08

    # Radius of convolution in "number grid cell". (2.5 is the standard value)
    conv_radius = 2.5 #2.5

    # Radius of deformable convolution in "number grid cell". Larger so that deformed kernel can spread out
    deform_radius = 6.0

    # Radius of the area of influence of each kernel point in "number grid cell". (1.0 is the standard value)
    KP_extent = 1.2

    # Behavior of convolutions in ('constant', 'linear', 'gaussian')
    KP_influence = 'gaussian' #linear

    # Aggregation function of KPConv in ('closest', 'sum')
    aggregation_mode = 'sum'

    # Choice of input features
    first_features_dim = 128
    in_features_dim = 1

    # Can the network learn modulations
    modulated = False

    # Batch normalization parameters
    use_batch_norm = True
    batch_norm_momentum = 0.02

    # Deformable offset loss
    # 'point2point' fitting geometry by penalizing distance from deform point to input points
    # 'point2plane' fitting geometry by penalizing distance from deform point to input point triplet (not implemented)
    deform_fitting_mode = 'point2point'
    deform_fitting_power = 1.0              # Multiplier for the fitting/repulsive loss
    deform_lr_factor = 0.1                  # Multiplier for learning rate applied to the deformations
    repulse_extent = 1.2                    # Distance of repulsion for deformed kernel points

    #####################
    # Training parameters
    #####################

    # Maximal number of epochs
    max_epoch = 500

    # Learning rate management
    learning_rate = 1e-4
    momentum = 0.98
    lr_decays = {i: 0.1 ** (1 / 150) for i in range(1, max_epoch)}
    grad_clip_norm = 100.0

    # Number of batch
    batch_num = 10

    # Number of steps per epochs
    epoch_steps = 100

    # Number of validation examples per epoch
    validation_size = 20

    # Number of epoch between each checkpoint
    checkpoint_gap = 10

    # Augmentations
    augment_scale_anisotropic = True
    augment_symmetries = [True, False, False]
    augment_rotation = 'vertical'
    augment_scale_min = 0.8
    augment_scale_max = 1.2
    augment_noise = 0.001
    augment_color = 0.8

    # The way we balance segmentation loss
    #   > 'none': Each point in the whole batch has the same contribution.
    #   > 'class': Each class has the same contribution (points are weighted according to class balance)
    #   > 'batch': Each cloud in the batch has the same contribution (points are weighted according cloud sizes)
    segloss_balance = 'class'
    #class_w = [0.02, 0.98]

    # Do we need to save convergence
    saving = True
    saving_path = None

In [4]:
config = APRPointCloudConfig()
train_dataset = APRPointCloudDataset(config, set = 'training')
validation_dataset = APRPointCloudDataset(config, set = 'validation')

training_sampler = APRPointCloudSampler(train_dataset)
validation_sampler = APRPointCloudSampler(validation_dataset)


Preparing KDTree for cloud t000, subsampled at 0.080
Shape of the loaded point cloud = (225567, 5)
6.3 MB loaded in 1.9s

Preparing KDTree for cloud t006, subsampled at 0.080
Shape of the loaded point cloud = (214399, 5)
6.0 MB loaded in 2.1s

Preparing KDTree for cloud t008, subsampled at 0.080
Shape of the loaded point cloud = (217210, 5)
6.1 MB loaded in 2.1s

Preparing KDTree for cloud t012, subsampled at 0.080
Shape of the loaded point cloud = (215785, 5)
6.0 MB loaded in 1.9s

Preparing KDTree for cloud t013, subsampled at 0.080
Shape of the loaded point cloud = (220526, 5)
6.2 MB loaded in 1.9s

Preparing KDTree for cloud t014, subsampled at 0.080
Shape of the loaded point cloud = (227671, 5)
6.4 MB loaded in 2.0s

Preparing KDTree for cloud t015, subsampled at 0.080
Shape of the loaded point cloud = (221920, 5)
6.2 MB loaded in 2.0s

Preparing KDTree for cloud t019, subsampled at 0.080
Shape of the loaded point cloud = (229155, 5)
6.4 MB loaded in 2.0s

Preparing KDTree for cl

In [5]:
training_loader = DataLoader(train_dataset,
                            collate_fn=APRPointCloudCollate,
                            sampler=training_sampler,)

validation_loader = DataLoader(validation_dataset,
                              collate_fn=APRPointCloudCollate,
                              sampler=training_sampler,)

training_sampler.calibration(training_loader)
validation_sampler.calibration(validation_loader)

net = KPFCNN(config, train_dataset.label_values, train_dataset.ignored_labels)
trainer = ModelTrainer(net, config)


Starting Calibration (use verbose=True for more details)
Calibration done in 0.0s


Starting Calibration (use verbose=True for more details)
Calibration done in 0.0s



In [ ]:
trainer.train(net, training_loader, validation_loader, config)

e000-i0000 => L=18.426 acc= 75% / t(ms):  44.9 1408.7 293.1)
e000-i0002 => L=17.212 acc= 69% / t(ms):  36.3 283.3 327.2)
e000-i0004 => L=17.714 acc= 50% / t(ms):  36.3 293.3 329.4)
e000-i0006 => L=18.419 acc= 78% / t(ms):  37.4 289.8 328.2)
e000-i0008 => L=17.189 acc= 75% / t(ms):  37.0 290.1 326.4)
e000-i0010 => L=17.287 acc= 75% / t(ms):  36.8 283.3 327.8)
e000-i0012 => L=17.468 acc= 70% / t(ms):  35.6 273.3 321.8)
e000-i0014 => L=17.586 acc= 83% / t(ms):  33.9 268.1 317.1)
e000-i0016 => L=17.446 acc= 73% / t(ms):  33.1 262.0 312.8)
e000-i0018 => L=17.148 acc= 78% / t(ms):  32.0 259.6 313.2)
e000-i0020 => L=17.399 acc= 92% / t(ms):  32.1 258.4 320.0)
e000-i0022 => L=16.935 acc= 94% / t(ms):  32.7 258.0 318.1)
e000-i0024 => L=17.525 acc= 72% / t(ms):  33.9 261.5 319.1)
e000-i0026 => L=16.875 acc= 89% / t(ms):  33.0 257.7 315.5)
e000-i0028 => L=16.717 acc= 69% / t(ms):  32.6 260.3 316.9)
e000-i0030 => L=16.909 acc= 88% / t(ms):  32.9 256.8 313.9)
e000-i0032 => L=17.171 acc= 92% / t(ms)

e002-i0022 => L=12.081 acc= 74% / t(ms):  32.4 298.6 336.8)
e002-i0024 => L=11.808 acc= 80% / t(ms):  32.4 298.2 345.6)
e002-i0026 => L=11.937 acc= 88% / t(ms):  33.0 305.2 345.0)
e002-i0028 => L=12.098 acc= 92% / t(ms):  33.0 302.1 344.7)
e002-i0030 => L=12.846 acc= 61% / t(ms):  34.4 306.5 347.6)
e002-i0032 => L=12.025 acc= 73% / t(ms):  36.4 310.5 350.4)
e002-i0034 => L=11.988 acc= 72% / t(ms):  34.4 296.5 339.9)
e002-i0036 => L=11.565 acc= 72% / t(ms):  33.1 294.3 332.2)
e002-i0038 => L=12.002 acc= 89% / t(ms):  32.5 286.3 325.5)
e002-i0040 => L=11.899 acc= 90% / t(ms):  31.3 276.1 323.6)
e002-i0042 => L=11.829 acc= 81% / t(ms):  31.1 279.3 319.1)
e002-i0044 => L=11.632 acc= 96% / t(ms):  30.4 277.1 314.3)
e002-i0046 => L=11.468 acc= 79% / t(ms):  29.6 271.2 310.5)
e002-i0048 => L=12.140 acc= 73% / t(ms):  29.4 271.0 311.6)
e002-i0050 => L=12.049 acc= 87% / t(ms):  29.6 281.7 320.5)
e002-i0052 => L=11.909 acc= 83% / t(ms):  30.3 288.9 325.3)
e002-i0054 => L=11.529 acc= 89% / t(ms):

e004-i0040 => L=12.024 acc= 89% / t(ms):  28.0 261.1 310.9)
e004-i0042 => L=11.990 acc= 85% / t(ms):  28.1 261.9 311.0)
e004-i0044 => L=12.358 acc= 78% / t(ms):  27.6 264.1 311.5)
e004-i0046 => L=11.938 acc= 92% / t(ms):  29.5 268.2 317.2)
e004-i0048 => L=11.569 acc= 83% / t(ms):  30.7 276.0 325.0)
e004-i0050 => L=12.389 acc= 87% / t(ms):  33.3 281.6 330.0)
e004-i0052 => L=12.070 acc= 91% / t(ms):  34.3 284.1 332.5)
e004-i0054 => L=12.135 acc= 61% / t(ms):  33.9 291.3 334.2)
e004-i0056 => L=12.096 acc= 92% / t(ms):  34.4 300.9 337.5)
e004-i0058 => L=11.676 acc= 89% / t(ms):  34.1 297.3 339.0)
e004-i0060 => L=11.626 acc= 87% / t(ms):  33.3 294.4 347.3)
e004-i0062 => L=11.608 acc= 98% / t(ms):  33.6 300.7 347.7)
e004-i0064 => L=11.995 acc= 85% / t(ms):  33.7 299.5 348.1)
e004-i0066 => L=11.964 acc= 75% / t(ms):  36.9 297.2 348.6)
e004-i0068 => L=11.958 acc= 75% / t(ms):  35.8 302.2 346.2)
e004-i0070 => L=12.369 acc= 75% / t(ms):  35.5 301.2 346.1)
e004-i0072 => L=12.062 acc= 76% / t(ms):

e006-i0058 => L=11.638 acc= 72% / t(ms):  35.1 296.4 348.2)
e006-i0060 => L=11.995 acc= 95% / t(ms):  35.1 307.5 348.1)
e006-i0062 => L=12.044 acc= 69% / t(ms):  34.8 307.1 348.8)
e006-i0064 => L=11.555 acc= 90% / t(ms):  34.2 310.3 351.9)
e006-i0066 => L=11.639 acc= 94% / t(ms):  33.6 311.1 354.6)
e006-i0068 => L=11.873 acc= 72% / t(ms):  34.0 314.1 352.1)
e006-i0070 => L=11.574 acc= 85% / t(ms):  34.0 311.5 358.3)
e006-i0072 => L=11.897 acc= 76% / t(ms):  34.0 310.2 364.4)
e006-i0074 => L=11.632 acc= 81% / t(ms):  33.9 307.9 361.6)
e006-i0076 => L=11.568 acc= 77% / t(ms):  33.4 302.8 357.0)
e006-i0078 => L=11.212 acc= 95% / t(ms):  32.8 307.8 353.6)
e006-i0080 => L=11.756 acc= 80% / t(ms):  32.5 304.6 351.7)
e006-i0082 => L=11.628 acc= 89% / t(ms):  32.7 300.8 349.4)
e006-i0084 => L=11.233 acc= 90% / t(ms):  33.1 304.1 349.4)
e006-i0086 => L=11.621 acc= 94% / t(ms):  33.0 301.2 349.5)
e006-i0088 => L=11.635 acc= 94% / t(ms):  33.0 304.0 348.5)
e006-i0090 => L=11.313 acc= 88% / t(ms):

e008-i0060 => L=11.087 acc= 89% / t(ms):  53.9 390.1 444.0)
e008-i0062 => L=11.245 acc= 88% / t(ms):  52.7 389.2 440.1)
e008-i0064 => L=11.375 acc= 94% / t(ms):  53.4 387.4 437.0)
e008-i0066 => L=11.302 acc= 79% / t(ms):  54.9 391.2 445.1)
e008-i0068 => L=11.499 acc= 86% / t(ms):  55.2 391.9 440.5)
e008-i0070 => L=11.222 acc= 98% / t(ms):  54.9 391.4 446.1)
e008-i0072 => L=11.274 acc= 91% / t(ms):  53.0 378.9 431.2)
e008-i0074 => L=11.296 acc= 83% / t(ms):  50.8 374.8 417.7)
e008-i0076 => L=11.424 acc= 92% / t(ms):  48.4 365.8 411.3)
e008-i0078 => L=11.312 acc= 85% / t(ms):  45.7 353.8 393.9)
e008-i0080 => L=10.909 acc= 94% / t(ms):  42.8 339.4 378.6)
e008-i0082 => L=12.112 acc= 66% / t(ms):  42.1 332.6 373.9)
e008-i0084 => L=11.294 acc= 92% / t(ms):  43.5 342.2 384.2)
e008-i0086 => L=11.071 acc= 96% / t(ms):  46.3 355.8 389.0)
e008-i0088 => L=11.488 acc= 70% / t(ms):  47.1 364.2 397.4)
e008-i0090 => L=11.623 acc= 92% / t(ms):  49.6 370.7 408.9)
e008-i0092 => L=11.546 acc= 90% / t(ms):

...

e030-i0070 => L=11.079 acc= 71% / t(ms):  37.3 316.4 369.9)
e030-i0072 => L=11.122 acc= 80% / t(ms):  37.4 316.2 369.2)
e030-i0074 => L=10.988 acc= 95% / t(ms):  38.3 318.5 369.7)
e030-i0076 => L=10.645 acc= 93% / t(ms):  40.3 325.0 379.0)
e030-i0078 => L=10.952 acc= 84% / t(ms):  39.4 319.2 371.9)
e030-i0080 => L=11.023 acc= 92% / t(ms):  41.5 317.3 366.6)
e030-i0082 => L=10.992 acc= 91% / t(ms):  41.1 317.8 368.0)
e030-i0084 => L=11.248 acc= 77% / t(ms):  42.3 319.5 368.1)
e030-i0086 => L=10.958 acc= 90% / t(ms):  41.8 319.4 366.1)
e030-i0088 => L=10.945 acc= 98% / t(ms):  41.9 319.4 364.5)
e030-i0090 => L=10.954 acc= 99% / t(ms):  41.1 316.5 363.1)
e030-i0092 => L=11.150 acc= 83% / t(ms):  39.9 315.1 364.2)
e030-i0094 => L=11.410 acc= 63% / t(ms):  39.3 318.9 365.8)
e030-i0096 => L=11.021 acc= 96% / t(ms):  39.4 317.1 365.8)
e030-i0098 => L=10.971 acc= 93% / t(ms):  38.9 315.7 368.7)
Validation : 25.0% (timings : 11.96 35.94)
Validation : 55.0% (timings : 22.50 62.35)
Validation : 8

e032-i0082 => L=10.998 acc= 95% / t(ms):  37.8 315.2 353.9)
e032-i0084 => L=10.705 acc= 89% / t(ms):  37.1 310.6 354.4)
e032-i0086 => L=11.143 acc=100% / t(ms):  36.9 313.6 356.0)
e032-i0088 => L=11.315 acc=100% / t(ms):  37.2 314.6 362.1)
e032-i0090 => L=10.958 acc= 87% / t(ms):  36.5 314.7 360.6)
e032-i0092 => L=10.440 acc= 94% / t(ms):  35.5 310.3 356.8)
e032-i0094 => L=11.121 acc= 98% / t(ms):  35.8 306.7 354.7)
e032-i0096 => L=11.208 acc= 93% / t(ms):  35.3 307.9 356.2)
e032-i0098 => L=11.258 acc= 79% / t(ms):  36.2 308.1 363.1)
Validation : 25.0% (timings : 12.54 36.51)
Validation : 50.0% (timings : 22.25 61.76)
Validation : 75.0% (timings : 30.39 81.73)
Validation : 105.0% (timings : 35.83 98.16)
Validation : 130.0% (timings : 39.77 109.05)
Validation : 160.0% (timings : 42.24 120.20)
Validation : 190.0% (timings : 43.53 126.77)
Validation : 220.0% (timings : 44.88 129.67)
Validation : 250.0% (timings : 47.06 131.94)
Validation : 280.0% (timings : 47.38 135.59)
Validation : 310.

e034-i0090 => L=11.671 acc= 95% / t(ms):  33.5 302.2 341.7)
e034-i0092 => L=11.298 acc= 91% / t(ms):  32.2 293.1 335.0)
e034-i0094 => L=11.047 acc= 94% / t(ms):  33.0 292.3 333.5)
e034-i0096 => L=11.304 acc= 98% / t(ms):  33.0 294.1 344.6)
e034-i0098 => L=10.766 acc= 96% / t(ms):  32.9 298.7 343.8)
Validation : 25.0% (timings : 12.26 36.72)
Validation : 55.0% (timings : 22.01 66.21)
Validation : 85.0% (timings : 28.89 85.51)
Validation : 110.0% (timings : 33.99 100.59)
Validation : 135.0% (timings : 39.43 111.83)
Validation : 165.0% (timings : 42.65 120.14)
Validation : 195.0% (timings : 44.69 126.72)
Validation : 225.0% (timings : 45.01 130.41)
Validation : 255.0% (timings : 45.41 131.83)
Validation : 280.0% (timings : 46.67 135.61)
Validation : 310.0% (timings : 46.74 135.17)
Validation : 340.0% (timings : 47.43 136.35)
Validation : 370.0% (timings : 47.51 136.09)
Validation : 400.0% (timings : 48.36 136.74)
Validation : 430.0% (timings : 49.03 137.91)
Validation : 455.0% (timings : 

Validation : 115.0% (timings : 34.81 99.70)
Validation : 145.0% (timings : 38.07 111.38)
Validation : 175.0% (timings : 41.19 119.91)
Validation : 205.0% (timings : 42.96 125.78)
Validation : 235.0% (timings : 44.30 128.45)
Validation : 265.0% (timings : 46.49 131.98)
Validation : 295.0% (timings : 46.45 135.60)
Validation : 325.0% (timings : 47.37 137.42)
Validation : 355.0% (timings : 48.29 138.33)
Validation : 385.0% (timings : 48.16 139.05)
Validation : 415.0% (timings : 48.94 140.50)
Validation : 445.0% (timings : 48.77 141.98)
Validation : 475.0% (timings : 49.58 143.45)
Confusion Matrix
****************
[[146773.58     3660.4216]
 [ 13520.748   50204.254 ]]
[0.89520794 0.74503136]
APRPointCloud mean IoU = 82.0%
e037-i0000 => L=11.339 acc= 99% / t(ms): 19537.7 327.2 359.7)
e037-i0002 => L=11.151 acc= 80% / t(ms):  33.0 312.0 341.5)
e037-i0004 => L=10.942 acc= 80% / t(ms):  33.7 308.0 341.6)
e037-i0006 => L=11.617 acc= 87% / t(ms):  34.6 310.8 343.6)
e037-i0008 => L=10.632 acc= 86

Validation : 420.0% (timings : 49.91 140.40)
Validation : 450.0% (timings : 49.25 140.55)
Validation : 480.0% (timings : 49.98 142.47)
Confusion Matrix
****************
[[142265.38    8168.621]
 [ 13705.936  50019.066]]
[0.86673224 0.6957372 ]
APRPointCloud mean IoU = 78.1%
Shape of the loaded point cloud = (214159, 5)
e039-i0000 => L=10.783 acc= 96% / t(ms): 22017.1 307.2 349.1)
e039-i0002 => L=11.344 acc= 89% / t(ms):  42.9 301.5 414.8)
e039-i0004 => L=10.949 acc= 91% / t(ms):  41.2 301.5 403.0)
e039-i0006 => L=10.585 acc= 99% / t(ms):  39.8 298.4 399.3)
e039-i0008 => L=11.022 acc= 80% / t(ms):  38.6 298.1 390.0)
e039-i0010 => L=10.880 acc= 88% / t(ms):  37.8 296.9 383.6)
e039-i0012 => L=11.572 acc= 94% / t(ms):  37.5 297.9 392.3)
e039-i0014 => L=11.052 acc= 97% / t(ms):  37.0 297.7 392.1)
e039-i0016 => L=11.268 acc= 99% / t(ms):  37.0 300.1 384.5)
e039-i0018 => L=11.629 acc= 97% / t(ms):  36.4 301.9 378.0)
e039-i0020 => L=11.342 acc= 94% / t(ms):  36.9 302.4 371.5)
e039-i0022 => L=1